<p style="font-weight:bold;"> <span style="font-size: 36px"> Reimport of Transactional Data Test</span> </p>

In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

#  Definitions

## Args

Create:
- Import args for current year (2021)
- Import args for previous year (2020)
- Partition by Reporting Node (using details from most recent args)

In [0]:
var args = new ImportArgs("CH", 2021, 3, Periodicity.Quarterly, null, null);
var previousArgs = new ImportArgs("CH", 2020, 12, Periodicity.Quarterly, null, null);

var partitionByReportingNode = new PartitionByReportingNode { Id = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNode>(args)), ReportingNode = args.ReportingNode };

## DataNodeParameter

Create 1 data node parameter

In [0]:
var dtr13IParameter = new InterDataNodeParameter{ Partition = partitionByReportingNode.Id, DataNode = "DTR1.3", Year = args.Year, Month = args.Month, LinkedDataNode = "DT1.4", ReinsuranceCoverage = 1d };

## DataNodeState

Create 7 data node states:
- dt11StatePrevious
- dtr11StatePrevious
- dt11State
- dtr11State
- dtr13StatePrevious
- dt13State
- dt14State

In [0]:
var dt11StatePrevious = new DataNodeState { DataNode = "DT1.1", State = State.Active, Year = previousArgs.Year, Month = previousArgs.Month, Partition = partitionByReportingNode.Id };

var dtr11StatePrevious = new DataNodeState { DataNode = "DTR1.1", State = State.Active, Year = previousArgs.Year, Month = previousArgs.Month, Partition = partitionByReportingNode.Id };

var dt11State = new DataNodeState { DataNode = "DT1.1", State = State.Active, Year = args.Year, Month = args.Month, Partition = partitionByReportingNode.Id };

var dtr11State = new DataNodeState { DataNode = "DTR1.1", State = State.Active, Year = args.Year, Month = args.Month, Partition = partitionByReportingNode.Id };

var dtr13StatePrevious = new DataNodeState { DataNode = "DTR1.3", State = State.Active, Year = previousArgs.Year, Month = previousArgs.Month, Partition = partitionByReportingNode.Id };

var dt13State = new DataNodeState { DataNode = "DT1.3", State = State.Active, Year = args.Year, Month = args.Month, Partition = partitionByReportingNode.Id };

var dt14State = new DataNodeState { DataNode = "DT1.4", State = State.Active, Year = args.Year, Month = args.Month, Partition = partitionByReportingNode.Id };

# Reimport Openings for DataNode at Inception

Create openings for a data node at inception (DT1.1) and import more than once.

## Disposable Workspace

In [0]:
var ws = Workspace.CreateNew();
ws.Initialize(x=>x.FromSource(DataSource).DisableInitialization<DataNodeState>());

await ws.UpdateAsync<DataNodeState>(new DataNodeState [] { dt11State, dtr11State });

## Import Phase

In [0]:
var openings = @"
@@Main,,,,
ReportingNode,Year,Month,,
CH,2021,3,,
@@Opening,,,,
DataNode,EstimateType,AmountType,AccidentYear,Value
DT1.1,C,,,72.2
DT1.1,AA,PR,,-1.5
DT1.1,OA,PR,,1.5
";

In [0]:
await Import.FromString(openings).WithFormat(ImportFormats.Opening).WithTarget(ws).ExecuteAsync()

Collect the IfrsVariable and RawVariable workspace queries after import.

In [0]:
var importQueries = new { 
    IfrsVariables = ws.Query<IfrsVariable>().ToArray(), 
    RawVariables = ws.Query<RawVariable>().ToArray()
};

## Reimport Phase

In [0]:
await Import.FromString(openings).WithFormat(ImportFormats.Opening).WithTarget(ws).ExecuteAsync()

Collect the IfrsVariable and RawVariable workspace queries after reimport.

In [0]:
var reimportQueries = new { 
    IfrsVariables = ws.Query<IfrsVariable>().ToArray(), 
    RawVariables = ws.Query<RawVariable>().ToArray()
};

## Assertions

Check for IfrsVariables

In [0]:
importQueries.IfrsVariables.Should().BeEquivalentTo(reimportQueries.IfrsVariables);

In [0]:
importQueries.IfrsVariables.Count().Should().Be(7);

In [0]:
reimportQueries.IfrsVariables.Count().Should().Be(7);

Check for RawVariables

In [0]:
importQueries.RawVariables.Count().Should().Be(0);

In [0]:
reimportQueries.RawVariables.Count().Should().Be(0);

# Reimport Actuals for Both: DataNode at Inception and DataNode and Not at Inception

Create an actuals file that contains data for both a data node at inception (DT1.3) and a data node not at inception (DT1.1). 

Import this file more than once.

## Disposable Workspace

In [0]:
var ws = Workspace.CreateNew();
ws.Initialize(x=>x.FromSource(DataSource).DisableInitialization<DataNodeState>());

await ws.UpdateAsync<DataNodeState>(new DataNodeState [] { dt11StatePrevious, dtr11StatePrevious, dt13State });

## Import Phase

In [0]:
var actuals = @"
@@Main,,,,,
ReportingNode,Year,Month,,,
CH,2021,3,,,
@@Actual,,,,,
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,PR,A,,90
DT1.1,CF,ICO,A,,-6
DT1.1,CF,NIC,A,,-70
DT1.1,CF,ACA,A,,-10
DT1.1,CF,AEA,A,,-5
DT1.3,CF,PR,A,,90
DT1.3,CF,ICO,A,,-6
DT1.3,CF,NIC,A,,-70
DT1.3,CF,ACA,A,,-10
DT1.3,CF,AEA,A,,-5";

In [0]:
await Import.FromString(actuals).WithFormat(ImportFormats.Actual).WithTarget(ws).ExecuteAsync()

Collect the IfrsVariable and RawVariable workspace queries after import.

In [0]:
var importQueries = new { 
    IfrsVariables = ws.Query<IfrsVariable>().ToArray(), 
    RawVariables = ws.Query<RawVariable>().ToArray()
};

##  Reimport Phase

In [0]:
await Import.FromString(actuals).WithFormat(ImportFormats.Actual).WithTarget(ws).ExecuteAsync();

Collect the IfrsVariable and RawVariable workspace queries after reimport.

In [0]:
var reimportQueries = new { 
    IfrsVariables = ws.Query<IfrsVariable>().ToArray(), 
    RawVariables = ws.Query<RawVariable>().ToArray()
};

## Assertions

Check for IfrsVariables

In [0]:
importQueries.IfrsVariables.Should().BeEquivalentTo(reimportQueries.IfrsVariables);

In [0]:
importQueries.IfrsVariables.Count().Should().Be(20);

In [0]:
reimportQueries.IfrsVariables.Count().Should().Be(20);

Check for RawVariables

In [0]:
importQueries.RawVariables.Count().Should().Be(0);

In [0]:
reimportQueries.RawVariables.Count().Should().Be(0);

# Import Cashflows for Pre-exisitng Reinsurance, Openings for Linked Insurance and Cashflows for the Reinsurance

Consider one GRIC (DTR1.3) that is linked to one GIC (DT1.4). The GRIC already existed the previous year, while the GIC is at inception.
- Import Cashflows for the GRIC in the previous period.
- Import Openings for the GIC in the current period.
- Import Cashflows for the GRIC in the current period.

## Disposable Workspace

In [0]:
var ws = Workspace.CreateNew();
ws.Initialize(x=>x.FromSource(DataSource).DisableInitialization<DataNodeState>().DisableInitialization<DataNodeParameter>().DisableInitialization<InterDataNodeParameter>());

await ws.UpdateAsync<DataNodeState>(new DataNodeState [] { dtr13StatePrevious, dt14State });
await ws.UpdateAsync<InterDataNodeParameter>(new InterDataNodeParameter [] { dtr13IParameter });

## Import Sequence Phase

### Import Cashflows for the GRIC in the Previous Period

In [0]:
var cashflows = @"
@@Main,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ReportingNode,Year,Month,Scenario,,,,,,,,,,,,,,,,,,,,,,,,,,
CH,2020,12,,,,,,,,,,,,,,,,,,,,,,,,,,,
@@Cashflow,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DTR1.3,PR,BE,CL,C,,100,0,0,100,0,0,100,0,0,100,0,0,0,100,0,0,100,0,0,100,0,0,100,0";

In [0]:
await Import.FromString(cashflows).WithFormat(ImportFormats.Cashflow).WithTarget(ws).ExecuteAsync()

### Assertions

In [0]:
ws.Query<RawVariable>().Where(x => (x.EstimateType == "BE" && x.AmountType == "PR" && x.AocType == "CL" && x.Novelty == "C")).Count().Should().Be(1);

In [0]:
ws.Query<RawVariable>().Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "L" && x.EstimateType == "BE" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "C" && x.EstimateType == "BE" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "N" && x.EstimateType == "BE" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -400d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "C" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == 399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "L" && x.EstimateType == "F" && x.AocType == "AM" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == 1d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "C" && x.AocType == "AM" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "L" && x.EstimateType == "BE" && x.AocType == "EOP" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "C" && x.EstimateType == "BE" && x.AocType == "EOP" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "N" && x.EstimateType == "BE" && x.AocType == "EOP" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -400)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Count().Should().Be(9);

### Import Openings for the GIC in the Current Period

In [0]:
var openings = @"
@@Main
ReportingNode,Year,Month,,
CH,2021,3,,
@@Opening
DataNode,EstimateType,AmountType,AccidentYear,Value
DT1.4,C,,,1";

In [0]:
await Import.FromString(openings).WithFormat(ImportFormats.Opening).WithTarget(ws).ExecuteAsync()

### Assertions (previous period)

In [0]:
await ws.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(previousArgs)

In [0]:
ws.Query<RawVariable>().Where(x => (x.EstimateType == "BE" && x.AmountType == "PR" && x.AocType == "CL" && x.Novelty == "C")).Count().Should().Be(1);

In [0]:
ws.Query<RawVariable>().Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "L" && x.EstimateType == "BE" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "C" && x.EstimateType == "BE" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "N" && x.EstimateType == "BE" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -400d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "C" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == 399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "L" && x.EstimateType == "F" && x.AocType == "AM" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == 1d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "C" && x.AocType == "AM" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "L" && x.EstimateType == "BE" && x.AocType == "EOP" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "C" && x.EstimateType == "BE" && x.AocType == "EOP" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "N" && x.EstimateType == "BE" && x.AocType == "EOP" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -400d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Count().Should().Be(9);

### Assertions (current period)

In [0]:
await ws.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(args)

In [0]:
ws.Query<RawVariable>().Count().Should().Be(0);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "C" && x.AocType == "EOP" && x.DataNode == "DT1.4" && x.Novelty == "C" && x.Values[0] == 1.0012476630625269)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "C" && x.AocType == "BOP" && x.DataNode == "DT1.4" && x.Novelty == "I" && x.Values[0] == 1d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "C" && x.AocType == "IA" && x.DataNode == "DT1.4" && x.Novelty == "I" && x.Values[0] == 0.0012476630625268825)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Count().Should().Be(3);

### Import Cashflows for the GRIC in the Current Period

In [0]:
var cashflows = @"
@@Main,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ReportingNode,Year,Month,Scenario,,,,,,,,,,,,,,,,,,,,,,,,,,
CH,2021,3,,,,,,,,,,,,,,,,,,,,,,,,,,,
@@Cashflow,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DTR1.3,PR,BE,CL,C,,100,0,0,100,0,0,100,0,0,100,0,0,0,100,0,0,100,0,0,100,0,0,100,0";

In [0]:
await Import.FromString(cashflows).WithFormat(ImportFormats.Cashflow).WithTarget(ws).ExecuteAsync()

### Assertions (previous period)

In [0]:
await ws.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(previousArgs)

In [0]:
ws.Query<RawVariable>().Where(x => (x.EstimateType == "BE" && x.AmountType == "PR" && x.AocType == "CL" && x.Novelty == "C")).Count().Should().Be(1);

In [0]:
ws.Query<RawVariable>().Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "L" && x.EstimateType == "BE" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "C" && x.EstimateType == "BE" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "N" && x.EstimateType == "BE" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -400d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "C" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == 399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "L" && x.EstimateType == "F" && x.AocType == "AM" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == 1d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "C" && x.AocType == "AM" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "L" && x.EstimateType == "BE" && x.AocType == "EOP" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "C" && x.EstimateType == "BE" && x.AocType == "EOP" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "N" && x.EstimateType == "BE" && x.AocType == "EOP" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -400d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Count().Should().Be(9);

### Assertions (current period)

In [0]:
await ws.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(args)

In [0]:
ws.Query<RawVariable>().Where(x => (x.EstimateType == "BE" && x.AmountType == "PR" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C")).Count().Should().Be(1);

In [0]:
ws.Query<RawVariable>().Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "L" && x.EstimateType == "BE" && x.AmountType == "PR" && x.DataNode == "DTR1.3" && x.AocType == "BOP" && x.Values[0] == -399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "C" && x.EstimateType == "BE" && x.AmountType == "PR" && x.DataNode == "DTR1.3" && x.AocType == "BOP" && x.Values[0] == -399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "N" && x.EstimateType == "BE" && x.AmountType == "PR" && x.DataNode == "DTR1.3" && x.AocType == "BOP" && x.Values[0] == -400)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "C" && x.AocType == "EOP" && x.DataNode == "DT1.4" && x.Novelty == "C" && x.Values[0] == 1.0012476630625269)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "C" && x.AocType == "BOP" && x.DataNode == "DT1.4" && x.Novelty == "I" && x.Values[0] == 1d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "C" && x.AocType == "IA" && x.DataNode == "DT1.4" && x.Novelty == "I" && x.Values[0] == 0.0012476630625268825)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "L" && x.EstimateType == "BE" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -399.08544669484894)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "C" && x.EstimateType == "BE" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] ==  -397.72149807405714)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "N" && x.EstimateType == "BE" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -400d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "C" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == 399.08544669484894)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "L" && x.EstimateType == "BE" && x.AocType == "BOP" && x.DataNode == "DTR1.3" && x.Novelty == "I" && x.Values[0] ==  -399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "C" && x.EstimateType == "BE" && x.AocType == "BOP" && x.DataNode == "DTR1.3" && x.Novelty == "I" && x.Values[0] == -399.63392949885525)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "N" && x.EstimateType == "BE" && x.AocType == "BOP" && x.DataNode == "DTR1.3" && x.Novelty == "I" && x.Values[0] == -400)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "L" && x.EstimateType == "F" && x.AocType == "AM" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == 1d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "C" && x.AocType == "AM" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -399.08544669484894)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "L" && x.EstimateType == "BE" && x.AocType == "EOP" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -698.8857378808717)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "C" && x.EstimateType == "BE" && x.AocType == "EOP" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -697.2233135002064 )).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "N" && x.EstimateType == "BE" && x.AocType == "EOP" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -700d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "L" && x.EstimateType == "BE" && x.AocType == "IA" && x.DataNode == "DTR1.3" && x.Novelty == "I" && x.Values[0] == -0.1663616871675563)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "C" && x.EstimateType == "BE" && x.AocType == "IA" && x.DataNode == "DTR1.3" && x.Novelty == "I" && x.Values[0] == -0.1663616871675563)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "N" && x.EstimateType == "BE" && x.AocType == "CF" && x.DataNode == "DTR1.3" && x.Novelty == "I" && x.Values[0] == 100d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "C" && x.EstimateType == "BE" && x.AocType == "CF" && x.DataNode == "DTR1.3" && x.Novelty == "I" && x.Values[0] == 100d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "L" && x.EstimateType == "BE" && x.AocType == "CF" && x.DataNode == "DTR1.3" && x.Novelty == "I" && x.Values[0] == 100d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "C" && x.EstimateType == "BE" && x.AocType == "YCU" && x.DataNode == "DTR1.3" && x.Novelty == "I" && x.Values[0] ==  0.2984757598735541)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Count().Should().Be(21);

# Import Actuals for Pre-existing Reinsurance, Openings for Linked Insurance and Cashflows for the Reinsurance

Consider one GRIC (DTR1.3) that is linked to one GIC (DT1.4). The GRIC already existed the previous year, while the GIC is at inception.
- Import Actuals for the GRIC in the previous period.
- Import Openings for the GIC in the current period.
- Import Cashflows for the GRIC in the current period.

## Disposable Workspace

In [0]:
var ws = Workspace.CreateNew();
ws.Initialize(x=>x.FromSource(DataSource).DisableInitialization<DataNodeState>().DisableInitialization<DataNodeParameter>().DisableInitialization<InterDataNodeParameter>());

await ws.UpdateAsync<DataNodeState>(new DataNodeState [] { dtr13StatePrevious, dt14State });
await ws.UpdateAsync<InterDataNodeParameter>(new InterDataNodeParameter [] { dtr13IParameter });

## Import Sequence Phase

### Import Actuals for the GRIC in the Previous Period

In [0]:
var actuals = @"
@@Main,,,,,
ReportingNode,Year,Month,,,
CH,2020,12,,,
@@Actual,,,,,
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DTR1.3,CF,PR,AA,,1";

In [0]:
await Import.FromString(actuals).WithFormat(ImportFormats.Actual).WithTarget(ws).ExecuteAsync()

### Assertions

In [0]:
ws.Query<RawVariable>().Count().Should().Be(0);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "AA" && x.AmountType == "PR" && x.AocType == "CF" && x.Novelty == "C" && x.Values[0] == 1d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "AA" && x.AmountType == "PR" && x.AocType == "EOP" && x.Novelty == "C" && x.Values[0] == 1d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Count().Should().Be(2);

### Import Openings for the GIC in the Current Period

In [0]:
var openings = @"
@@Main
ReportingNode,Year,Month,,
CH,2021,3,,
@@Opening
DataNode,EstimateType,AmountType,AccidentYear,Value
DT1.4,C,,,1";

In [0]:
await Import.FromString(openings).WithFormat(ImportFormats.Opening).WithTarget(ws).ExecuteAsync()

### Import Cashflows for the GRIC in the Current Period

In [0]:
var cashflows = @"
@@Main,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ReportingNode,Year,Month,Scenario,,,,,,,,,,,,,,,,,,,,,,,,,,
CH,2021,3,,,,,,,,,,,,,,,,,,,,,,,,,,,
@@Cashflow,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DTR1.3,PR,BE,CL,C,,100,0,0,100,0,0,100,0,0,100,0,0,0,100,0,0,100,0,0,100,0,0,100,0";

In [0]:
await Import.FromString(cashflows).WithFormat(ImportFormats.Cashflow).WithTarget(ws).ExecuteAsync()

### Assertions (previous period)

In [0]:
await ws.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(previousArgs)

In [0]:
ws.Query<RawVariable>().Count().Should().Be(0);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "AA" && x.AmountType == "PR" && x.AocType == "CF" && x.Novelty == "C" && x.Values[0] == 1d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "AA" && x.AmountType == "PR" && x.AocType == "EOP" && x.Novelty == "C" && x.Values[0] == 1d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Count().Should().Be(2);

### Assertions (current period)

In [0]:
await ws.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(args)

In [0]:
ws.Query<RawVariable>().Where(x => (x.EstimateType == "BE" && x.AmountType == "PR" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C")).Count().Should().Be(1);

In [0]:
ws.Query<RawVariable>().Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "C" && x.AocType == "EOP" && x.DataNode == "DT1.4" && x.Novelty == "C" && x.Values[0] ==  1.0012476630625269)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "C" && x.AocType == "BOP" && x.DataNode == "DT1.4" && x.Novelty == "I" && x.Values[0] == 1d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "C" && x.AocType == "IA" && x.DataNode == "DT1.4" && x.Novelty == "I" && x.Values[0] == 0.0012476630625268825)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "AA" && x.AmountType == "PR" && x.AocType == "EOP" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == 1d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "AA" && x.AmountType == "PR" && x.AocType == "BOP" && x.DataNode == "DTR1.3" && x.Novelty == "I" && x.Values[0] == 1d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "L" && x.EstimateType == "BE" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] ==  -698.8857378808717)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "C" && x.EstimateType == "BE" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -697.2233135002064)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "N" && x.EstimateType == "BE" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -700)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "C" && x.AocType == "CL" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] ==  698.8857378808717)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "L" && x.EstimateType == "F" && x.AocType == "AM" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == 1d)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "C" && x.AocType == "AM" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -698.8857378808717)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "L" && x.EstimateType == "BE" && x.AocType == "EOP" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] ==  -698.8857378808717 )).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "C" && x.EstimateType == "BE" && x.AocType == "EOP" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -697.2233135002064)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EconomicBasis == "N" && x.EstimateType == "BE" && x.AocType == "EOP" && x.DataNode == "DTR1.3" && x.Novelty == "C" && x.Values[0] == -700)).Count().Should().Be(1);

In [0]:
ws.Query<IfrsVariable>().Count().Should().Be(14);

# Import Modified Openings with CSM Deletion and LC Addition for DataNode at Inception

Create openings for a data node at inception (DT1.1) and import. 

Import again with the difference that CSM has been deleted and LC (for the same DataNode) has been added.

## Disposable Workspace

In [0]:
var ws = Workspace.CreateNew();
ws.Initialize(x=>x.FromSource(DataSource).DisableInitialization<DataNodeState>());

await ws.UpdateAsync<DataNodeState>(new DataNodeState [] { dt11State, dtr11State });

In [0]:
var openings = @"
@@Main,,,,
ReportingNode,Year,Month,,
CH,2021,3,,
@@Opening,,,,
DataNode,EstimateType,AmountType,AccidentYear,Value
DT1.1,C,,,72.2
DT1.1,AA,PR,,-1.5
DT1.1,OA,PR,,1.5";

## Import Phase

In [0]:
await Import.FromString(openings).WithFormat(ImportFormats.Opening).WithTarget(ws).ExecuteAsync()

## Assertions

In [0]:
ws.Query<RawVariable>().Count().Should().Be(0);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "C")).Count().Should().Be(3);

In [0]:
ws.Query<IfrsVariable>().Count().Should().Be(7);

## Reimport Phase

In [0]:
var openings = @"
@@Main,,,,
ReportingNode,Year,Month,,
CH,2021,3,,
@@Opening,,,,
DataNode,EstimateType,AmountType,AccidentYear,Value
DT1.1,L,,,72.2
DT1.1,AA,PR,,-1.5
DT1.1,OA,PR,,1.5";

In [0]:
await Import.FromString(openings).WithFormat(ImportFormats.Opening).WithTarget(ws).ExecuteAsync()

## Assertions

In [0]:
ws.Query<RawVariable>().Count().Should().Be(0);

In [0]:
ws.Query<IfrsVariable>().Where(x => (x.EstimateType == "L")).Count().Should().Be(3);

In [0]:
ws.Query<IfrsVariable>().Count().Should().Be(7);